# Merging Data
Steps to consider:
* Want to use weighted similarity
    * multiply the similarity score of each ingredient pair by its corresponding weight based on the relative positions of the ingredients in their respective lists
    * Sum up the weighted similarity scores across all ingredient pairs to obtain an overall similarity score between the two products
    * Assign higher weights to ingredients that are closer together, and higher weights if they are closer to 1 in the list
* Need to match ingredients to their place in the ingredient_info data set

In [25]:
import pandas as pd
import re as re
import numpy as np
from IPython.display import display, clear_output

In [3]:
ingredients = pd.read_csv('data/ingredient_info_processed.csv')
products = pd.read_csv('data/skincare_products_processed.csv')

In [4]:
def ingredient_in_INCI(inci, ingredient_name):
    '''
    Takes two string arguments: an official INCI name, and an ingredient
    (as listed in the product dataframe). Returns True if every word in
    the ingredient name can be found in the INCI name. 

    This is a helper function used in get_shortest_INCI().
    '''

    # Removes whitespace and punctuation and puts text in lowercase
    cleaned_inci_text = re.sub(r'[^\w\s]', ' ', inci).lower() 
    cleaned_ingredient_text = re.sub(r'[^\w\s]', ' ', ingredient_name).lower()

    # Creates a set from the cleaned text which contains all of the words
    # in the cleaned up string
    words_in_inci = set(cleaned_inci_text.split())
    words_in_name = set(cleaned_ingredient_text.split())

    # Using set comparison: if the words in ingredient name are a subset
    # of the words in the inci name, return True. Otherwise, return False.
    return (words_in_name <= words_in_inci)



def get_shortest_INCI(df, ingredient_name):
    ''' 
    Takes the ingredients dataframe (df) and an ingredient name
    as arguments, then returns the matching row from the dataframe if
    a match is found. Otherwise, returns an empty row.
    '''

    # Uses ingredient_in_INCI to map and filter the df by whether the  given 
    # ingredient name matches an INCI name in the df. 
    filtered_df = \
        df[df['INCI name'].apply(lambda x: ingredient_in_INCI(x, ingredient_name))]
    
    # If no matches are found, try searching again but with removing words 
    # that are surrounded by parenthesis.
    if filtered_df.size == 0:
        ingredient_name_clean = re.sub(r'\([^)]*\)', '', ingredient_name)

        filtered_df = \
            df[df['INCI name'].apply(lambda x: ingredient_in_INCI(x, ingredient_name_clean))]
        
        # If there are still no matches, return the empty dataframe
        if filtered_df.size == 0:
            return filtered_df

    # If there are multiple matches, return only the row with the
    # shortest INCI name. Ensures only one row is returned.  
    shortest_index = filtered_df['INCI name'].str.len().idxmin()
    return filtered_df.loc[[shortest_index]]


In [26]:
def combine_ingredients_and_info(product_df, ingredient_df, copy = True):
  '''
  Takes the products and ingredients dataframes as arguments, then adds INCI name, 
  COSING Ref No, Description, and Function of matching ingredients from the
  ingredients dataframe to the products dataframe.
  If copy = True, returns a copy of the dataframe. Otherwise, it returns the
  modified product_df.
  '''
  def combine(product_df, ingredient_df):
    # Initializes the empty new columns
    product_df['INCI name'] = ""
    product_df['COSING Ref No'] = np.nan
    product_df['Ingredient Description'] = ""
    product_df['Ingredient Function'] = ""

    for index, row in product_df.iterrows():
      # Gets the ingredient name and finds the matching row in ingredient_df
      # using helper function get_shortest_INCI
      ingredient_name = str(row['Ingredient'])
      INCI_details = get_shortest_INCI(ingredient_df, ingredient_name)

      message = f"Adding row {index}..."
      display(message)

      # If a match is found, then fill in the row with the matching info.
      # Otherwise, the row will remain at its default (empty) values.
      if INCI_details.size > 0:
        product_df.at[index, 'INCI name'] = \
          INCI_details['INCI name'].iloc[0]
        
        product_df.at[index, 'COSING Ref No'] = \
          INCI_details['COSING Ref No'].iloc[0]
        
        product_df.at[index, 'Ingredient Description'] = \
          INCI_details['Chem/IUPAC Name / Description'].iloc[0]
        
        product_df.at[index, 'Ingredient Function'] = \
          INCI_details['Function'].iloc[0]
      
      clear_output(wait=True)

    return product_df

  # If copy is True, then create a deep copy and return the copy. Otherwise,
  # return the modified df.
  if copy:
    product_df_copy = product_df.copy()
    return combine(product_df_copy, ingredient_df)
  
  return combine(product_df, ingredient_df)

In [27]:
combined_df = combine_ingredients_and_info(products, ingredients, copy=False)

'Adding row 45815...'

In [34]:
combined_df.drop(columns=["Combination", "Dry", "Oily", "Normal", "Sensitive"]).head()

,Label,Brand,Name,Price,Rank,Ingredient_Placement,Ingredient,INCI name,COSING Ref No,Ingredient Description,Ingredient Function
0,Moisturizer,LA MER,Crème de la Mer,175,4.1,1,Algae (Seaweed) Extract,ALGAE EXTRACT,54290.0,Algae Extract is an extract of various species...,"FRAGRANCE, HUMECTANT, ORAL CARE, SKIN CONDITIO..."
1,Moisturizer,LA MER,Crème de la Mer,175,4.1,2,Mineral Oil,HYDROGENATED MINERAL OIL,95058.0,Hydrogenated Mineral Oil is the end product of...,SKIN PROTECTING
2,Moisturizer,LA MER,Crème de la Mer,175,4.1,3,Petrolatum,PETROLATUM,79504.0,Petrolatum. A complex combination of hydrocarb...,"ANTISTATIC, SKIN CONDITIONING - EMOLLIENT"
3,Moisturizer,LA MER,Crème de la Mer,175,4.1,4,Glycerin,GLYCERIN,34040.0,Glycerine ;Glycerol (INN); Glycerol (RIFM); G...,"DENATURANT, HAIR CONDITIONING, HUMECTANT, ORAL..."
4,Moisturizer,LA MER,Crème de la Mer,175,4.1,5,Isohexadecane,ISOHEXADECANE,34654.0,"Hydrocarbons, C4, 1,3-butadiene-free, polym., ...","SKIN CONDITIONING, SKIN CONDITIONING - EMOLLIE..."


In [35]:
combined_df.to_csv("data/skincare_products_merged.csv", index=False)